# Website Traffic Analysis

In this project we will perform click rate analysis to find the most sought out links in a website. This analysis will try to answer questions such as



1.   How does clickrate distribute among different links
2.   Per day analysis of clickrates for each link
3.   Is there any correlation between clickrates and previews.




In [120]:
import pandas as pd

from scipy import stats

In [2]:
traffic_df = pd.read_csv('traffic.csv')
traffic_df.head()

,event,date,country,city,artist,album,track,isrc,linkid
0,click,2021-08-21,Saudi Arabia,Jeddah,Tesher,Jalebi Baby,Jalebi Baby,QZNWQ2070741,2d896d31-97b6-4869-967b-1c5fb9cd4bb8
1,click,2021-08-21,Saudi Arabia,Jeddah,Tesher,Jalebi Baby,Jalebi Baby,QZNWQ2070741,2d896d31-97b6-4869-967b-1c5fb9cd4bb8
2,click,2021-08-21,India,Ludhiana,Reyanna Maria,So Pretty,So Pretty,USUM72100871,23199824-9cf5-4b98-942a-34965c3b0cc2
3,click,2021-08-21,France,Unknown,"Simone & Simaria, Sebastian Yatra",No Llores Más,No Llores Más,BRUM72003904,35573248-4e49-47c7-af80-08a960fa74cd
4,click,2021-08-21,Maldives,Malé,Tesher,Jalebi Baby,Jalebi Baby,QZNWQ2070741,2d896d31-97b6-4869-967b-1c5fb9cd4bb8


In [3]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226278 entries, 0 to 226277
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   event    226278 non-null  object
 1   date     226278 non-null  object
 2   country  226267 non-null  object
 3   city     226267 non-null  object
 4   artist   226241 non-null  object
 5   album    226273 non-null  object
 6   track    226273 non-null  object
 7   isrc     219157 non-null  object
 8   linkid   226278 non-null  object
dtypes: object(9)
memory usage: 15.5+ MB


In [4]:
traffic_df.shape

(226278, 9)

# Per day Views

In [5]:
traffic_df['event'].value_counts()

pageview    142015
click        55732
preview      28531
Name: event, dtype: int64

In [6]:
traffic_df['date'] = pd.to_datetime(traffic_df['date'])
traffic_df.dtypes

event              object
date       datetime64[ns]
country            object
city               object
artist             object
album              object
track              object
isrc               object
linkid             object
dtype: object

In [24]:
x1 = traffic_df[traffic_df['event'] == 'pageview'].groupby(traffic_df.date).agg({'event':'count'})

In [25]:
x2 = traffic_df.groupby(traffic_df.date).agg({'event':'count'})

Lets calculate the percentage of events that were pageviews out of total views per day. 

In [28]:
x1/x2 * 100

,event
date,
2021-08-19,63.250474
2021-08-20,62.681754
2021-08-21,62.638265
2021-08-22,62.605338
2021-08-23,62.553677
2021-08-24,62.656700
2021-08-25,62.892896


# Countries that clicks originated from and the count
 Lets determine top 5 countries from which the clicks originated

In [55]:
df = traffic_df[traffic_df.event == 'click'].groupby('country')['event'].count().reset_index()

In [52]:
df.head()

,country,event
0,Albania,76
1,Algeria,129
2,American Samoa,1
3,Angola,36
4,Anguilla,2


In [51]:
df.sort_values('event',ascending=False).iloc[:5,:]

,country,event
84,India,11609
192,United States,9533
158,Saudi Arabia,9495
62,France,4054
87,Iraq,1836


# Determine overall clickrate

ClickThroughRate (CTR) is the number of clicks a page is recieving. So in this case that would be clicks/pageviews


In [97]:
df_pageviews = traffic_df[traffic_df.event == 'pageview'].groupby('linkid')['event'].count().reset_index()

In [104]:
df_pageviews.rename(columns={'event':'pageview'},inplace=True)

**To get the CTR we will join both the dataframes on linkid column such that we can perform column arithmatic to obtain CTR**

In [105]:
df_click = traffic_df[traffic_df.event == 'click'].groupby('linkid')['event'].count().reset_index()
df_click.rename(columns={'event':'click'},inplace=True)

In [108]:
new_df = pd.merge(left=df_pageviews,right=df_click,on='linkid')
new_df.head()

,linkid,pageview,click
0,00126b32-0c35-507b-981c-02c80d2aa8e7,2,2
1,004b9724-abca-5481-b6e9-6148a7ca00a5,1,1
2,0063a982-41cd-5629-96d0-e1c4dd72ea11,3,2
3,006af6a0-1f0d-4b0c-93bf-756af9071c06,36,8
4,00759b81-3f04-4a61-b934-f8fb3185f4a0,4,3


In [109]:
new_df['CTR'] = new_df.click/new_df.pageview

In [110]:
new_df

,linkid,pageview,click,CTR
0,00126b32-0c35-507b-981c-02c80d2aa8e7,2,2,1.000000
1,004b9724-abca-5481-b6e9-6148a7ca00a5,1,1,1.000000
2,0063a982-41cd-5629-96d0-e1c4dd72ea11,3,2,0.666667
3,006af6a0-1f0d-4b0c-93bf-756af9071c06,36,8,0.222222
4,00759b81-3f04-4a61-b934-f8fb3185f4a0,4,3,0.750000
...,...,...,...,...
2248,ffd8d5a7-91bc-48e1-a692-c26fca8a8ead,84,29,0.345238
2249,fff38ca0-8043-50cd-a5f1-f65ebb7105c5,1,1,1.000000
2250,fff84c0e-90a1-59d8-9997-adc909d50e16,1,1,1.000000
2251,fffc17a7-f935-5d3e-bd3e-d761fd80d479,2,1,0.500000


# Find if there exists correlation between clicks,previews

To calculate the correlation between two continuos values such as clicks,previews we can use pearson correlation value.

To calculate coreelation between categorical and continuos values we can use point-beserial correlation

In [115]:
df_preview = traffic_df[traffic_df.event == 'preview'].groupby('linkid')['event'].count().reset_index()
df_click = traffic_df[traffic_df.event == 'click'].groupby('linkid')['event'].count().reset_index()
#df_click.rename(columns={'event':'click'},inplace=True)
#df_preview.rename(columns={'event':'pageview'},inplace=True)

In [124]:
df_click

,linkid,click
0,00126b32-0c35-507b-981c-02c80d2aa8e7,2
1,004b9724-abca-5481-b6e9-6148a7ca00a5,1
2,0063a982-41cd-5629-96d0-e1c4dd72ea11,2
3,006af6a0-1f0d-4b0c-93bf-756af9071c06,8
4,00759b81-3f04-4a61-b934-f8fb3185f4a0,3
...,...,...
2250,ffd8d5a7-91bc-48e1-a692-c26fca8a8ead,29
2251,fff38ca0-8043-50cd-a5f1-f65ebb7105c5,1
2252,fff84c0e-90a1-59d8-9997-adc909d50e16,1
2253,fffc17a7-f935-5d3e-bd3e-d761fd80d479,1


In [122]:
df_preview

,linkid,pageview
0,006af6a0-1f0d-4b0c-93bf-756af9071c06,11
1,00759b81-3f04-4a61-b934-f8fb3185f4a0,1
2,00829040-ee01-4409-966d-d67c7965144a,9
3,00de7566-f014-4d20-8616-82e4dea45b88,1
4,0211bf4d-0e9d-46c2-889c-f97872280820,141
...,...,...
490,fd647d87-4c7a-4dd1-92c6-0e7dd7fdf4f1,101
491,fe24ad63-e347-40db-8e75-94e573bd3d9b,1
492,fe8a3fcd-7beb-4db0-8494-5238e1e1cab3,57
493,fe8f7a23-be9d-49a6-b9b5-d26823c3f911,22


In [118]:
df_merged = pd.merge(left=df_preview,right=df_click,on='linkid')
df_merged.sort_values(by='pageview',ascending=False)

,linkid,pageview,click
81,2d896d31-97b6-4869-967b-1c5fb9cd4bb8,5974,9692
141,522da5cc-8177-4140-97a7-a84fdb4caf1c,1605,2109
418,e849515b-929d-44c8-a505-e7622f1827e9,1571,2198
350,c2c876ab-b093-4750-9449-6b4913da6af3,1001,1429
279,97105aff-3e40-4c2e-bfeb-563b55e8f5dd,849,1111
...,...,...,...
386,d6e0167f-42dc-4621-9034-0b1c6a34a95c,1,1
123,4856aef8-f48a-4882-9744-bc3c5c3f0335,1,1
305,a70732bc-444f-4013-a5d8-772ea853a28b,1,6
382,d547e535-817d-4f1f-9feb-9e25fd600bd2,1,1


In [119]:
df_merged.corr()

<ipython-input-119-5bcae2bf3446>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_merged.corr()


,pageview,click
pageview,1.000000,0.993422
click,0.993422,1.000000


In [121]:
stats.pearsonr(df_merged.pageview,df_merged.click)

PearsonRResult(statistic=0.9934224704248961, pvalue=0.0)